In [1]:
import pandas as pd
# input data
coro_data = pd.read_csv('coro_data.csv')
coro_data.head(10)

<ipython-input-1-55398f23cfa8>:3: DtypeWarning: Columns (23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  coro_data = pd.read_csv('coro_data.csv')


,Host,Virus,HostTaxID,VirusTaxID,HostNCBIResolved,VirusNCBIResolved,ICTVRatified,HostGenus,HostFamily,HostOrder,...,PMID,ReleaseYear,ReleaseMonth,ReleaseDay,CollectionYear,CollectionMonth,CollectionDay,AssocID,DatabaseDOI,NCBIAccession
0,NaN,NaN,10114,11118,True,True,False,rattus,muridae,rodentia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,33554,693996,True,True,False,NaN,NaN,carnivora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,371929,694002,True,True,False,pitangus,tyrannidae,passeriformes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,49442,694002,True,True,False,rhinolophus,rhinolophidae,chiroptera,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,9397,11118,True,True,False,NaN,NaN,chiroptera,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,9397,693996,True,True,False,NaN,NaN,chiroptera,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,9787,694002,True,True,False,NaN,NaN,perissodactyla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,9822,693996,True,True,False,sus,suidae,artiodactyla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,9895,694002,True,True,False,NaN,bovidae,artiodactyla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# get coroviridae data with NCBIAccession number
id_accession_data = coro_data[['VirusTaxID','NCBIAccession']]
# remove na
id_accession_data.dropna(inplace=True)

# construct a dictionary, the key of which is ID number and value is NCBIAccession
dic_ID_Accession = {}

for key,value in zip(id_accession_data['VirusTaxID'],id_accession_data['NCBIAccession']):
    process_value = []
    # split the accession number by comma
    for i in value.split(','):
        # remove space
        process_value.append(i.replace(" ",""))
    dic_ID_Accession[key] = dic_ID_Accession.get(key, []) + process_value


<ipython-input-2-890880d44c55>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_accession_data.dropna(inplace=True)


In [3]:
# construct two lists to conserve id and accession
IDs = []
Accessions = []

# construct a two columns data frame
for key, value in dic_ID_Accession.items():
    n_value_count = len(value)
    IDs = IDs + [key]*n_value_count
    Accessions = Accessions + value

coro_id_accession_csv_data = pd.DataFrame(IDs, Accessions)
coro_id_accession_csv_data = coro_id_accession_csv_data.reset_index() # reset the column name
coro_id_accession_csv_data.columns = ['Accession', 'VirusID']

In [4]:
# remove duplication
coro_id_accession_csv_data = coro_id_accession_csv_data.drop_duplicates(['Accession', 'VirusID'])
# conserve data into a excel table
coro_id_accession_csv_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv',index=None)

In [5]:
# input coroviridae data with VirusID and Accession
import pandas as pd
coro_id_accession = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv')

# forcedly convert data type from many types to string 
coro_id_accession["Accession"] = coro_id_accession["Accession"].astype('str')

In [6]:
# extract the number that represents the NCBI id of url
import re
number = []
accession_data = coro_id_accession["Accession"]

for i in accession_data:
    pattern = re.compile(r"^[0-9]\d+")
    number = number + re.findall(pattern, i)


In [7]:
# get the accession number through the number id
import requests
import re

def get_fasta_info(id):
    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409'.format(id)
    fasta_info = requests.get(fasta_url,timeout=(10,45)).text
    return fasta_info

number_accession = {}
failured_number_id = []

# using WHILE can improve the readablity of code
# the step is designed to convert GI numbers to Accession number, which consumes a lot of time
for i in range(12):
    for id in number[i*100:(i+1)*100]:
        try:
            fasta_info = get_fasta_info(id)
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
        except:
            failured_number_id.append(id)
            print("The {} id numbers are failured to convert to accession".format(id))
            fasta_info = get_fasta_info(failured_number_id[0])
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
            print("The {} id numbers are successed to convert to accession".format(id))
            failured_number_id.pop(0)


The 87244987 id numbers are failured to convert to accession
The 87244987 id numbers are successed to convert to accession
The 182702146 id numbers are failured to convert to accession
The 182702146 id numbers are successed to convert to accession
The 226316432 id numbers are failured to convert to accession
The 226316432 id numbers are successed to convert to accession


In [8]:
# check the length
len(number_accession.keys())

1195

In [16]:
# the step, conserving the data into a csv file, is unnecessary
numberID_accession_data = pd.DataFrame.from_dict(number_accession, orient='index')
numberID_accession_data = numberID_accession_data.reset_index()
numberID_accession_data.columns = ['GI', 'Accession']
numberID_accession_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/GI_Accession.csv',index=None)

In [18]:
# use dictary data format to accomplish the step is to replace GI by Accession
coro_id_accession = coro_id_accession.replace(number_accession)

In [19]:
# conserve data
coro_id_accession.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/final_Acce_VirusID.csv',index=None)